In [8]:
# 전처리 코드

import pandas as pd
from sklearn.preprocessing import StandardScaler
import os

# 여러 CSV 파일을 읽어 전처리하는 함수
def preprocess_multiple_csv(file_paths, pose_name):
    # 모든 데이터를 저장할 빈 DataFrame 생성
    combined_df = pd.DataFrame()
    scaler = StandardScaler()

    for file_path in file_paths:
        # CSV 파일을 DataFrame으로 로딩
        temp_df = pd.read_csv(file_path)

        # 필요한 경우 여기서 각 파일에 대한 추가적인 전처리를 수행

        # 모든 DataFrame을 하나로 합치기
        combined_df = pd.concat([combined_df, temp_df], ignore_index=True)

    # 모든 데이터에 대해 'x', 'y', 'z' 열을 정규화
    combined_df[['x', 'y', 'z']] = scaler.fit_transform(combined_df[['x', 'y', 'z']])

    # 자세 이름을 라벨로 추가
    combined_df['pose_name'] = pose_name

    return combined_df

# 모든 CSV 파일 경로
directory_path = '/content/multiple_csv'  # CSV 파일이 저장된 디렉토리 경로
file_names = os.listdir(directory_path)
file_paths = [os.path.join(directory_path, file_name) for file_name in file_names if file_name.endswith('.csv')]

# 파일 경로 리스트와 자세 이름을 함수에 전달하여 전처리
pose_name = "Warrior_Pose"
preprocessed_data = preprocess_multiple_csv(file_paths, pose_name)

# 전처리된 데이터를 새로운 CSV 파일로 저장
preprocessed_data.to_csv('Warrior_pose_data.csv', index=False)

# ... 이전의 전처리 코드 ...

# 여기에 교차 검증 코드를 추가합니다.
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# 모델 인스턴스 생성
classifier = RandomForestClassifier(random_state=42)

# 특성(features)과 라벨(labels) 분리
X = preprocessed_data.drop('pose_name', axis=1)  # 특성만 포함
y = preprocessed_data['pose_name']  # 라벨만 포함

# 5-겹 교차 검증 수행
scores = cross_val_score(classifier, X, y, cv=5, scoring='accuracy')

# 성능 결과 출력
print("Accuracy scores for each fold:")
print(scores)
print("\nAverage cross-validation score:")
print(scores.mean())



Accuracy scores for each fold:
[1. 1. 1. 1. 1.]

Average cross-validation score:
1.0


In [10]:
# 훈련용 vs 테스트용 비교하는 코드

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# 훈련용 데이터 불러오기
train_csv_path = '/content/Warrior_pose_data.csv'  # 훈련 데이터 파일 경로
train_data = pd.read_csv(train_csv_path)

# 테스트용 데이터 불러오기
test_csv_path = '/content/compare2.csv'  # 테스트 데이터 파일 경로
test_data = pd.read_csv(test_csv_path)

# 훈련 데이터에서 특성과 라벨 분리
X_train = train_data.drop('pose_name', axis=1)
y_train = train_data['pose_name']

# 테스트 데이터에서 특성과 라벨 분리
X_test = test_data.drop('pose_name', axis=1)
y_test = test_data['pose_name']

# 모델 훈련
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)

# 테스트 데이터로 성능 평가
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

Warrior_Pose       1.00      1.00      1.00        33

    accuracy                           1.00        33
   macro avg       1.00      1.00      1.00        33
weighted avg       1.00      1.00      1.00        33

